In [1]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
data_url = 'df_'
train_data_full = TabularDataset(f'{data_url}train.csv')
train_data_full.head()

,Dst Port,Protocol,Fwd Pkt Len Std,Bwd Pkt Len Min,Flow Byts/s,Fwd IAT Mean,Bwd IAT Tot,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,...,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Std,Active Max,Active Min,Idle Max,Idle Min,Label
0,0.506211,0.352941,0.026531,0.0,3.702185e-07,0.898484,0.000149,0.000113,0.000131,0.000018,...,0.003632,0.003235,0.000004,0.666667,0.0,0.0,0.0,0.0,0.0,Benign
1,0.006760,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.979767,0.000000,0.000000,0.416667,0.0,0.0,0.0,0.0,0.0,Benign
2,0.123295,0.352941,0.000000,0.0,0.000000e+00,0.897541,0.000000,0.000000,0.000000,0.000000,...,0.003922,0.000000,0.000000,0.416667,0.0,0.0,0.0,0.0,0.0,Botnet
3,0.000320,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.410217,0.000015,0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,Brute-force
4,0.000320,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.410217,0.000015,0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,Brute-force


In [4]:
# 定义转换为二分类问题的函数
def to_binary_classification(data, positive_class):
    # 将正类标签保持不变，其他所有标签转换为负类标签
    data['Label'] = data['Label'].apply(lambda x: x if x == positive_class else 'Malicious')
    return data

# 应用这个函数到训练和测试数据集
positive_class_label = 'Benign'  # 这里假设'Benign'是正类标签
train_data_binary = to_binary_classification(train_data_full, positive_class_label)

# 接下来，您可以使用 train_data_binary 和 test_data_binary 来训练和评估模型


In [5]:
train_data_sampled = train_data_binary.sample(frac=0.1, random_state=42)

In [6]:
label = 'Label'
train_data_sampled[label].describe()

count     362399
unique         2
top       Benign
freq      181719
Name: Label, dtype: object

In [13]:
save_path = 'model_save_path/'  # specifies folder to store trained models
predictor = TabularPredictor(label=label).fit(train_data_sampled)

No path specified. Models will be saved in: "AutogluonModels\ag-20240330_190849"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240330_190849"
=================== System

In [14]:
test_data_full = TabularDataset(f'{data_url}test.csv')
test_data_binary = to_binary_classification(test_data_full, positive_class_label)
test_data_sampled = test_data_full.sample(frac=0.1, random_state=42)

y_pred = predictor.predict(test_data_sampled.drop(columns=[label]))
y_pred.head()

Loaded data from: df_test.csv | Columns = 35 / 35 | Rows = 905998 -> 905998


109585    Malicious
287587    Malicious
542475    Malicious
233266       Benign
796602    Malicious
Name: Label, dtype: object

In [15]:
predictor.evaluate(test_data_sampled, silent=True)

{'accuracy': 0.9701986754966887,
 'balanced_accuracy': 0.9702593932245623,
 'mcc': 0.941527124425172,
 'roc_auc': 0.9923265575670724,
 'f1': 0.9695307739183425,
 'precision': 0.9943519825929955,
 'recall': 0.9459185695725894}

In [16]:
predictor.leaderboard(test_data_sampled)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.970287,0.969095,accuracy,0.206739,0.018511,3.850024,0.206739,0.018511,3.850024,1,True,4
1,WeightedEnsemble_L2,0.970199,0.969923,accuracy,1.457171,0.114519,31.839570,0.004001,0.004001,1.148232,2,True,14
2,XGBoost,0.969106,0.968543,accuracy,0.122020,0.013006,2.423921,0.122020,0.013006,2.423921,1,True,11
3,NeuralNetFastAI,0.968951,0.968267,accuracy,0.584513,0.030001,155.977308,0.584513,0.030001,155.977308,1,True,10
4,ExtraTreesEntr,0.968907,0.967163,accuracy,1.101411,0.069002,23.508619,1.101411,0.069002,23.508619,1,True,9
5,ExtraTreesGini,0.968709,0.967163,accuracy,0.961492,0.076519,23.331951,0.961492,0.076519,23.331951,1,True,8
6,LightGBMXT,0.968664,0.967715,accuracy,0.158517,0.015001,2.966745,0.158517,0.015001,2.966745,1,True,3
7,NeuralNetTorch,0.968642,0.968267,accuracy,0.411061,0.032000,147.048983,0.411061,0.032000,147.048983,1,True,12
8,LightGBMLarge,0.968102,0.965784,accuracy,0.023000,0.009999,0.908773,0.023000,0.009999,0.908773,1,True,13
9,CatBoost,0.968035,0.966060,accuracy,0.043510,0.007001,7.218129,0.043510,0.007001,7.218129,1,True,7
